# Data Analysis Project City Bike NYC

**Goals**<br>
This dataset contains a sample of bike trips from the City Bike system in New York City.
Each row represents one trip and includes information about the start and end stations, the duration, the
user type, and other contextual data like age, season, temperature, and weekday.
Your goal is to explore this dataset and extract insights through data analysis with Pandas.

You'll practice basic pandas operations (loading, exploring, cleaning, transforming, summarizing) and use descriptive statistics and simple visualizations to support your answers.

## 1. Dataset Exploration

Environment requirements: Jupyter, Python, ipython, 
pandas openpyxl

In [ ]:
%pip install pandas openpyxl
%pip install matplotlib

### What information does each column contain?

Each row contains a record of a trip or unit of service usage: a bicycle was collected by a user somewhere sometime, used for a certain period of time and returned. It also contains information about the user's demographics and whether or not they are enrolled service members.

Looking at the Dataframe we can see there are a few columns,
['Start Time', 'Stop Time', 'Start Station ID', 'Start Station Name',<br>
       'End Station ID', 'End Station Name', 'Bike ID', 'User Type',<br>
       'Birth Year', 'Age', 'Age Groups', 'Trip Duration',<br>
       'Trip_Duration_in_min', 'Month', 'Season', 'Temperature', 'Weekday'],<br>

```'Start Time'```, ```'Stop Time'```: Show when the bicycle was picked up and when it was returned.<br>
```'Start Station ID'```, ```'End Station ID'```: The IDs of stations where bicycles were collected and returned.<br>
```'Start Station Name'```, ```'End Station Name'```: The station names corresponding to the Station IDs (vid. infra.)<br>
```'Bike ID'```: the unique ID for the bike used for the trip.<br>
```'User Type'```: whether the user is a member or not.<br>
```'Birth Year'```, ```'Age'```, ```'Age Groups'```: user demographic information (vid. infra.)<br>
```'Trip Duration'```, ```'Trip_Duration_in_min'```: Time elapsed between bicycle collection and return. Available in seconds and minutes (vid. infra.)<br>
```'Month'```, ```'Season'```: Colums related to time of year. (NB. there appear to be records only for January through March.) <br>
```'Temperature'```: the only weather mesurement available in the dataset. This will likely hinder any advanced weather-related insight, as we have no
information about rain, snow, etc.<br>
```'Weekday'```: using this column we might know what age groups use bicycles more often, as well as test assumptions on current bicycle usage.<br><br>

There appears to be strong correlation among certain fields -- likely the result of calculated fields,

- ```'Birth Year'```, ```'Age'``` and ```'Age Groups'```
- ```'Trip Duration'``` and ```'Trip_Duration_in_min'``` (both are obtained from either ```'Start Time'``` or ```'Stop Time'``` as they are in full date format.)
- ```'Month'``` and ```'Season'```, ```'Weekday'``` (same as above.)

Also, there is some data duplication as there is no relational database, namely;

- ```'Start Station Name'``` and ```'End Station Name'``` will correspond to the same IDs (ie. ```'Start Station ID'``` or ```'End Station ID'```, respectively.)

In [ ]:
# Load Pandas dataframe

import pandas as pd

df = pd.read_excel('ny_citibikes_raw.xlsx', sheet_name='NYCitiBikes')

# Test df has loaded up
df.head()

In [ ]:
# List column names

print(df.columns)

### Are there missing or duplicated values?

We can see a single missing value in column **'End Station Name'** and *3,555 duplicate rows* across the dataset.

In [ ]:
# isna, drop, null values?

# Check for missing values
print(df.isna().sum())

# Check for duplicate rows
duplicate_rows = df.duplicated().sum()
print(f"\nNumber of duplicate rows: {duplicate_rows}")







We can either recover the End Station Name from the End Station ID, or just drop the row. Since this just affects one row and we don't know if the error might have corrupted other values it's probably best to just drop (delete) the row. 

In [ ]:
# Find missing row and drop it.
print("Row count:", len(df))
print("Locating the row with missing values...")
missing_row = df[df['End Station Name'].isna()]
print("Total number of missing rows located: ", missing_row.shape[0])

print("Deleting rows with missing values...")
df = df.dropna(subset=['End Station Name'])
print("Rows deleted")
print("New row count:", len(df))


Now let's delete the Duplicate Rows.

In [ ]:
# Drop duplicate rows and chech row totals.

print("Row count:", len(df))
print("Total duplicate rows:", duplicate_rows)
print("Expected rows after duplicate deletion:",(len(df) - duplicate_rows))
print("Deleting duplicate rows...")
df = df.drop_duplicates()
print("New row count:", len(df))

### What is the overall time span of the trips? (overall time range)


To calculate the overall time range, we find the oldest bicycle pickup timestamp, the most recent bicycle drop timestamp, and we subtract (newest - oldest).

In [ ]:
# Find latest and newest record and subtract

oldest_timestamp = df['Start Time'].min()
newest_timestamp = df['Stop Time'].max()
print("Oldest trip start Timestamp:\t", oldest_timestamp)
print("Newest trip stop Timestamp:\t", newest_timestamp)
print("Overall time range:\t\t", (newest_timestamp - oldest_timestamp))



## 2. Basic Statistics

### What is the average trip duration (in minutes)?

In [ ]:
avg_trip_mins = df['Trip_Duration_in_min'].mean()
print(f"Average trip duration: {avg_trip_mins:.2f} mins.")

### What is the minimum and maximum duration?

In [ ]:
min_trip_mins = df['Trip_Duration_in_min'].min()
max_trip_mins = df['Trip_Duration_in_min'].max()
print("Minimum trip duration:",min_trip_mins)
print("Maximum trip duration:",max_trip_mins)

### What are the most common start and end stations?

In [ ]:
top_start_station = df['Start Station Name'].value_counts().head(1)
top_end_station = df['End Station Name'].value_counts().head(1)

print(f"Top start station:\t{top_start_station.index[0]}. Trip count:\t{top_start_station.iloc[0]}")
print(f"Top end station:\t{top_end_station.index[0]}. Trip count:\t{top_end_station.iloc[0]}")

# station_name = top_station.index[0]
# trip_count = top_station.iloc[0]

# print(top_start_station)
# print(top_end_station)

 

## 3. Users and Demographics

### How many unique bikes were used?

We use ```.nunique()``` which is the unique count method. It returns the count of all distinct values of a a column, in this case *'Bike ID'*.

In [ ]:
unique_bikes = df["Bike ID"].nunique()
print("Total bikes used (unique): ", unique_bikes)

### What are the proportions of user types (Subscriber vs Customer)?

We need to tweak ```df.value_counts()``` to show our percentages.
- *value.counts()* by itself only returns each each class member's row count
- *value_counts(normalize=True)* divides said row count by total rows --> Decimal proportion
- *.mul(100)* multiplies by 100 to get the percentage

To list the user types and percentages we will generalise using the method ```.items()``` in an attempt to make the code more reusable.


In [ ]:
user_types = df['User Type'].value_counts(normalize=True).mul(100).round(2)


# There are only 2 user types in this case, so we could use
 # print(user_types.index[0], user_types.iloc[0])
 # print(user_types.index[1], user_types.iloc[1])

# But let's try and generalise to cater for any number of class members. 

print("User Type\tType percentage\n================================")

for user_type_name, user_type_percentage in user_types.items():
    print(f"{user_type_name}\t{user_type_percentage:.1f}%")


    


# print(user_types.items())


# print(user_types.iloc[0])
# print(user_types.iloc[1])


#print(f'''User Type\t\Type percentage
#      ================================
#      {user_types.index[1]})
#      '''


# print(f"Top start station:\t{top_start_station.index[0]}. Trip count:\t{top_start_station.iloc[0]}")
# print(f"Top end station:\t{top_end_station.index[0]}. Trip count:\t{top_end_station.iloc[0]}")


      

      


Let's make a simple pie chart.

In [ ]:
import matplotlib.pyplot as plt

# We'll use .index and .values as lists ("Duck typing")
labels = user_types.index
sizes = user_types.values

# Plot the chart 
#   ( autopct='%1.1f%%' --> to show percentages inside each slice)
plt.pie(sizes, labels=labels, autopct='%1.1f%%')
plt.title('Citi Bike user type by size (%)')
plt.show()

### What is the age distribution of the users? <br>Which age group uses the service the most?

As we can see below, the 35-44 age group is the most frequent service user group.

In [ ]:
# User age distribution
# Same as previous step, but we need to sort by user age as well

user_ages = df['Age Groups'].value_counts(normalize=True).mul(100).round(2)

# Sort by user age
user_ages = user_ages.sort_index()

print("User age bucket\t\tPercentage\n===================================")

for user_age_bucket, user_age_percentage in user_ages.items():
    print(f"{user_age_bucket}\t\t\t{user_age_percentage:.1f}%")


# labels = user_ages.index
# sizes = user_ages.values

To highlight the age group uses the service the most, we can just pick out the highest-percentage bucket with a one-liner like so;


In [ ]:
print("Most frequent service user group:", df['Age Groups'].value_counts().head(1).index[0])

Let's make a pie chart and see how it looks.

In [ ]:
# We'll use .index and .values as lists ("Duck typing")
labels = user_ages.index
sizes = user_ages.values

# Plot the chart 
#   ( autopct='%1.1f%%' --> to show percentages inside each slice)
plt.pie(sizes, labels=labels, autopct='%1.1f%%')
plt.title('Citi Bike service usage by Age Group (%)')
plt.show()

Since we have a few groups with small percentage values, they clump together in the pie chart. To add insult to injury such groups aren't in a continuous age range, so grouping them together wouldn't be the cleanest option in our opinion.<br>Let's try a **Bar Chart** instead.

In [ ]:
user_ages.plot(kind='bar')
plt.title('Citi Bike Usage by Age Group')
plt.ylabel('Percentage (%)')
plt.xticks(rotation=45)
plt.show()

Now it's much easier to spot the age groups with smaller values.<br>Another benefit of the bar chart over the pie chart is a tidy layout of age groups in ascending order (cf. x-axis) for quick reference, while the highest and lowest values are still easily spotted as they stand out visually.

## 4. Temporal Analysis

### How does the number of trips vary by weekday?

First we make sure all timestamps are healthy -- they are.

In [ ]:
# Check timestamp health
print("Bad Start Time Timestamps: ",(pd.to_datetime(df['Start Time'], errors='coerce').isna().sum()))
print("Bad Stop Time Timestamps: ",(pd.to_datetime(df['Stop Time'], errors='coerce').isna().sum()))

Next, we organise all trips by weekday and reindex.
<br>NB. Since the column 'Weekday' is a string, we need to manually provide the order of the days of the week.



In [ ]:
# 
trips_by_weekday = df['Weekday'].value_counts()

weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
trips_by_weekday = trips_by_weekday.reindex(weekday_order)

print(trips_by_weekday)


Let's plot another bar chart to show these values.

In [ ]:
# Bar Chart

trips_by_weekday.plot(kind='bar')
plt.title('Citi Bike Usage by Weekday (in # of trips)')
plt.ylabel('# of trips')
plt.xticks(rotation=45)
plt.show()


Now we can clearly see peak usage occurs on Wednesdays and Thursdays, and the trough on Saturdays and Sundays.

### Which month or season has the most rides?

We'll focus on months.

In [ ]:
trips_by_month = df['Month'].value_counts()
top_rides_month = trips_by_month.idxmax()

# Let's show the month name as well.
Month_Names = {1:'January', 2:'February', 3: 'March'}

print(f"The month No. with most rides is: {top_rides_month} ({Month_Names[top_rides_month]})")


### What time of day do most trips start?

Before we apply ```value_counts()``` we must first split by hour.<br>
Let's add a "Start Hour" column to the dataset for convenience.

In [ ]:
# Create a new column "Start Hour"
df['Start Hour'] = df['Start Time'].dt.hour

# Calculate top hour and trip count, show results
top_start_hour = df['Start Hour'].value_counts().idxmax()
top_start_hour_trips = df['Start Hour'].value_counts().max()
print(f"Most common hour (24 hr): {top_start_hour}.\nTotal trips in that hour: {top_start_hour_trips}.")

## 5. Geographic Analysis

### Which station pairs (start → end) appear most often?

We can create a new column to gather trips and just apply ```value.counts()``` to them.

In [ ]:
df['Trips'] = df['Start Station Name'] + ' > ' + df['End Station Name']
print("Most common trip routes\n=====================================")
df['Trips'].value_counts().head(5)


### Are there any stations that appear only as start or only as end stations?

We can save the unique values for each station type, the subtract the sets both ways.
Let's use a function to show the data.<br>
As we can see in the results below, there are no unique start station names

In [ ]:
start_stations = set(df['Start Station Name'].unique())
end_stations = set(df['End Station Name'].unique())

start_only_stations = start_stations - end_stations
end_only_stations = end_stations - start_stations



def show_stations(station_set, station_set_name):
    print(f"Showing records for {station_set_name}\n==============================================")
    if len(station_set) == 0:
        print("There are no records of: ", station_set_name, "\n")
    else:
        for station in station_set:
            print(station)
    


show_stations(start_only_stations, "start-only Stations")
show_stations(end_only_stations, "end-only Stations")

# for station in start_only_stations:
#    print(station)

# print(len(start_only_stations))
# print(len(end_only_stations))

## 6. Temperature and Duration

### Is there any visible relationship between temperature and trip duration?

This is called a *bivariate correlation analysis*, because it uses 2 variables -- in this case *temperature* and *trip duration*.
We can use the method ```df.corr()``` which calculates Pearson's correlation coefficient. The scale of results are shown like this;
- -1 or near: Strong negative correlation (ie. as one value increases the other decreases proportionally)
-  0 or near: No correlation (ie. no linear relationship)
- +1 or near: Strong positive correlation (ie. as one value increases the other increases proportionally)

In this particular instance, as we can see below, there is no correlation.

In [ ]:
# Pearson correlation coefficient

correlation_r = df['Trip_Duration_in_min'].corr(df['Temperature'])
print("The Pearson correlation coefficient is:", correlation_r)

### How does average trip duration vary by season?

We can start by looking at average trip per season.

In [ ]:
# Bar chart for trip duration average per Season

avg_duration_by_season = df.groupby('Season')['Trip_Duration_in_min'].mean()

plt.figure(figsize=(8, 6))

avg_duration_by_season.plot(
    kind='bar',
    color=['#008000', '#0000FF']
)

plt.title('Average Trip Duration by Season')
plt.xlabel('Season')
plt.ylabel('Average Trip Duration (Minutes)')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

There is hardly any difference to spot. Let's look at the mean and median values per season.

In [ ]:
seasonal_stats = df.groupby('Season')['Trip_Duration_in_min'].agg(['mean', 'median'])
print(seasonal_stats)

Looking at the mean and median values makes us none the wiser. Let's make a box plot to visualise the variance.

In [ ]:
# scatter

plt.figure(figsize=(10, 6))

# Generate the Box Plot
df.boxplot(
    column='Trip_Duration_in_min',
    by='Season',
    ax=plt.gca(),
    grid=True,
    patch_artist=True,
    boxprops=dict(facecolor='lightblue', color='blue'),
    medianprops=dict(color='red')
)

# Omit outliers --> not working!
showfliers=False

# Customizing the plot
plt.title('Trip Duration Distribution by Season (Box Plot)', fontsize=14)
plt.suptitle('') # Suppress pandas default title
plt.xlabel('Season', fontsize=12)
plt.ylabel('Trip Duration (Minutes)', fontsize=12)
plt.ylim(0, 40)  # Focus on the main body of data (0-30 mins)

# Tighten y-axis
plt.ylim(0, 20)

plt.show()

So the variance is not in the absolute terms of mean and median but in the frequency distribution of the trips. L

In [ ]:

# Define the quantiles (percentiles) to calculate: Q1, Median (Q2), Q3
quantiles = [0.25, 0.50, 0.75]

# 1. Group by season and calculate the quartiles
seasonal_quartiles = df.groupby('Season')['Trip_Duration_in_min'].quantile(quantiles).unstack()

# 2. Rename columns for clarity
seasonal_quartiles.columns = ['Q1 (25th)', 'Median (50th)', 'Q3 (75th)']

# 3. Calculate the Interquartile Range (IQR)
seasonal_quartiles['IQR'] = seasonal_quartiles['Q3 (75th)'] - seasonal_quartiles['Q1 (25th)']

print(seasonal_quartiles)

So now we can establish;
- The most frequent minimum trip duration is the same for both seasons: 4 minutes (Q1)
- Typical trip duration is the same for both seasons: 5 minutes (median value)
- The longest trips (Q3) are 1 minute longer in winter, which corresponds to a modest 12.5% increase relative to the 8-minute spring baseline.

## 7. Summary and Interpretation

The Citi Bike service is generally used as a utility and short-trip commuter service. We base this opinion on trip duration and variance, and on identifying the most frequent end trip Stations.

**Trip duration and variance**
- Typical trip durations are between 5 and 8-9 minutes, despite the service limit of 45 minutes per trip for members ([Source: Citi Bikes HP - Pricing](https://citibikenyc.com/pricing))
- Temperature has no bearing on trip duration
- Trip duration variance by Season is low

**End trip Stations**
The top 5 destination stations (W 45 St & 8 Ave, E 15 St & 3 Ave, JCBS Depot, Broadway & W 36 St, Warren St & Church St) are transit hubs, commercial centers, or university adjacent areas. Last mile commuting, typically from home to a nearby subway station is likely the most frequent use case.
